In [ ]:
# ===============================================================================================================#
# Copyright 2024 Infosys Ltd.                                                                          #
# Use of this source code is governed by Apache License Version 2.0 that can be found in the LICENSE file or at  #
# http://www.apache.org/licenses/                                                                                #
# ===============================================================================================================#

## Tool 03 - Prompt Engineering (Interactive)
To test different prompts on LLM

#### Import libraries

In [ ]:
import os
import requests
import re
import json
import tiktoken
from IPython.display import display, HTML, Markdown
from _internal_utils.q_n_a_visualizer import QnAVisualizer

#### Set environment variables
<div  style="line-height: 1;">
    <span style="color:Red"><b>NOTE:</b> The Pipeline uses environment variables which needs to be set by the developer.<br>
In production developer needs to set them as required.<br>
In this notebook you can provide them using the below code.<br>
To set or change the value please refer <i>installation.ipynb</i></span>
</div>

In [ ]:
%store -r CUSTOM_LLM_MIXTRAL_INFERENCE_URL
os.environ['CUSTOM_LLM_MIXTRAL_INFERENCE_URL']=CUSTOM_LLM_MIXTRAL_INFERENCE_URL

%store -r OPENAI_KEY
os.environ['OPENAI_KEY'] = OPENAI_KEY
%store -r OPENAI_SERVER_URL
os.environ['OPENAI_SERVER_URL']=OPENAI_SERVER_URL

os.environ["TIKTOKEN_CACHE_DIR"] = r"C:\MyProgramFiles\AI\models\tiktoken_encoding"

In [ ]:
def count_tokens(text):
    encoding = tiktoken.get_encoding("p50k_base")
    count = len(encoding.encode(text))
    return count

In [ ]:
class LlmService():

    def call_mixtral(self, question, remove_query=False):
        url = os.environ['CUSTOM_LLM_MIXTRAL_INFERENCE_URL']
        response = self.__invoke_api(url, question)
        if remove_query:
            response = self.__remove_query(response, question)
        return response

    def call_open_ai(self, question):
        url = os.environ['OPENAI_SERVER_URL']
        url+= "/openai/deployments/gpt4/chat/completions?api-version=2024-02-15-preview"
        headers = {
            "Content-Type": "application/json",
            "api-key": os.environ['OPENAI_KEY']
        }
#         data = {
#             "prompt": question,
#             "max_tokens": 100
#         }
        data={
          "messages": [
            {
              "role": "system",
              "content": "You are a helpful assistant."
            },
            {
              "role": "user",
              "content": question
            }
          ],
          "max_tokens": 2000,
          "stream": False
        }
        response = requests.post(url, headers=headers,
                                 data=json.dumps(data), timeout=180)
        print("response",response)
        print("response.json()",response.json())
        print("response.json()['choices'][0]['message']",response.json()['choices'][0]['message'])
        return response.json()['choices'][0]['message']["content"]

    ## Private ##

    def __invoke_api(self, url, query):
        # Set the JSON payload data as a Python dictionary
        json_payload = {
            "inputs": query,
            "parameters": {
                "max_new_tokens": 1000,
                "temperature": 0.1,
                "do_sample": True
            }
        }

        response = requests.post(
            url, json=json_payload, verify=False, timeout=180)
        print(response.json())
        return response.json()['generated_text']

    def __remove_query(self, answer, query):
        return answer[len(query):].strip()

In [ ]:
def show_widget(model_name:str, placeholders:dict):
    LLM_OPENAI = "openai_text-davinci-003"
    LLM_MIXTRAL = "Mixtral8X7B"
    if not model_name in [LLM_OPENAI, LLM_MIXTRAL]:
        return "Invalid model selected"
    qna_visualizer = QnAVisualizer(lhs_title='Prompt Template (LLM Request):', rhs_title='LLM Response:', 
                                   show_filter=False)
    if model_name == LLM_OPENAI:
        widget_title = "Open AI (Closed Model)"
    elif model_name == LLM_MIXTRAL:
        widget_title = "Mixtral8X7B (Open Model)"

    def form_submit_button_clicked(_):
        qna_visualizer.set_output_text('Fetching... Please wait...')
        input_text = qna_visualizer.get_input_text()
        
        for key, value in placeholders.items():
            key_str = "{" + key + "}"
            input_text = input_text.replace(key_str, value)        
        if model_name == LLM_OPENAI:
            model_output = LlmService().call_open_ai(input_text)
            print(model_output)
        elif model_name == LLM_MIXTRAL:
            model_output = LlmService().call_mixtral(input_text)
        model_output = model_output.strip()
        qna_visualizer.set_output_text(model_output)

    # Set callback functions
    qna_visualizer.set_token_counter_fn(count_tokens)
    qna_visualizer.on_form_submit_callback(form_submit_button_clicked)
    # Display UI
    
    display(HTML(f"<h2>{widget_title}</h2>"))
    qna_visualizer.show_ui()

In [ ]:
CONTEXT = """
In the depths of an abandoned train tunnel, John and Mark stumbled upon a mysterious machine. 
Its sleek, alien design hinted at a technology beyond their comprehension. With cautious excitement, 
they pressed buttons and pulled levers, inadvertently activating a time machine.

The tunnel vanished, replaced by a bustling 19th-century street. Wide-eyed, they realized 
the power they held. They explored eras, witnessing history's highs and lows. From ancient Rome 
to a futuristic metropolis, time unfurled before them.

But the machine's power waned. Panic set in as they realized they were trapped. 
They scrambled, trying to reverse their journey, but it was futile. The time machine 
blinked out, leaving them marooned in the past.
"""

QUESTION = "What did John and Mark discover?"

placeholders = {
    'context': CONTEXT,
    'question': QUESTION
    
}
SAMPLE_PROMPT_TEMPLATE_1 = """
Use the following pieces of context to answer the question at the end. 
If you don't know the answer or even doubtful a bit, just say that you don't know, 
don't try to make up an answer. Just give the shortest and most appropriate relevant answer 
to the question in proper json format with key as "answer". This json format should be followed 
even when answer is not found. 
{context}
Question: {question}
Helpful Answer:
"""
SAMPLE_PROMPT_TEMPLATE_2 = """
{context}
Question: {question}
Helpful Answer:
"""
print("---Sample prompt template 1---", SAMPLE_PROMPT_TEMPLATE_1)
print("---Sample prompt template 2---", SAMPLE_PROMPT_TEMPLATE_2)
show_widget("Mixtral8X7B", placeholders)

In [ ]:
show_widget("openai_text-davinci-003",placeholders)